In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt




In [31]:
df=pd.read_csv("C:\\Users\\srd9cxl\\OneDrive - Colruyt Group NV\\Desktop\\highbrow wines case\\Highbrow-wines-case-predictive-modelling-\\data_2016.csv")

In [32]:
df = df.dropna(subset=["bought_highbrow_wines"])
#.copy() to avoid SettingWithCopyWarning which is not relevant here

In [33]:
#lets separate targets early
y = df["bought_highbrow_wines"]
X = df.drop(columns=["bought_highbrow_wines"])


In [34]:
#lets fill msising collishop values with 0. reason: mssing= no evidence of digital trust
X["Collishop_customer"] = (
    X["Collishop_customer"]
    .map({"Y": 1, "N": 0})
    .fillna(0)
)


In [35]:
# df["cat_AP_STDR_WhiskyONLINE"].describe()
(df["cat_AP_STDR_WhiskyONLINE"] == 0).mean()

#spend columns are right skewed


np.float64(0.99669993399868)

In [36]:
#identifying spend columns
candidate_spend_cols = [
    col for col in X.columns
    if (
        col.startswith("cat_") or col.endswith("_rev")
    )
    and X[col].dtype != "object"
]
skew_summary = []

for col in candidate_spend_cols:
    zero_frac = (X[col] == 0).mean()
    max_val = X[col].max()
    median = X[col].median()

    skew_summary.append({
        "column": col,
        "zero_fraction": zero_frac,
        "median": median,
        "max": max_val
    })

skew_df = pd.DataFrame(skew_summary)
skew_df.sort_values("zero_fraction", ascending=False).head(15)


,column,zero_fraction,median,max
10,cat_ColruytMobile_Toestellen,0.999990,0.0,0.00
42,cat_nfokay,0.999670,0.0,14.97
0,cat_AP_STDR_PortoONLINE,0.999385,0.0,706.88
1,cat_AP_STDR_WhiskyONLINE,0.996700,0.0,878.84
34,cat_VNCVarkenRest,0.996345,0.0,108.46
23,cat_Textiel_Bedlinnen,0.995915,0.0,174.95
33,cat_VNCRest,0.994855,0.0,54.72
12,cat_EleKtroKeuken,0.990620,0.0,184.87
14,cat_Incontinentie_luiers,0.989655,0.0,1395.92
35,cat_VNCWildSteak,0.987145,0.0,310.50


In [38]:
X[candidate_spend_cols] = X[candidate_spend_cols].clip(lower=0)

In [ ]:
#semantic importance of spend columns
#Does €500 vs €50 actually mean 10× stronger behavior?
#For alcohol / gourmet / online revenue → YES
#For tech purchase / diapers / bedding → NO (presence matters more)

In [40]:
import numpy as np

X[candidate_spend_cols] = np.log1p(X[candidate_spend_cols])


In [ ]:
#quasi binary spends handled (quasi binary because of some small non-zero values)

X["cat_ColruytMobile_Toestellen_flag"] = (
    X["cat_ColruytMobile_Toestellen"] > 0
).astype(int)

X = X.drop(columns=["cat_ColruytMobile_Toestellen"])


In [42]:
#encode categorical variables
X = pd.concat([
    X,
    pd.get_dummies(X["SOW_type_colr"], prefix="SOW")
], axis=1)

X = X.drop(columns=["SOW_type_colr"])


In [43]:
#household typology
X = pd.concat([
    X,
    pd.get_dummies(X["HOUSEHOLDTYPOLOGY"], prefix="HH")
], axis=1)

X = X.drop(columns=["HOUSEHOLDTYPOLOGY"])


In [ ]:
assert X.isna().sum().sum() == 0



AssertionError: 

In [45]:
X.dtypes[X.dtypes == "object"]


Series([], dtype: object)

In [25]:
X.isna().sum().sum()
X.shape


(199996, 77)

In [28]:
# Find which columns have missing values
missing_cols = X.columns[X.isna().any()].tolist()
print("Columns with missing values:", missing_cols)

# Show rows with missing values
print("\nRows with missing values:")
print(X[X.isna().any(axis=1)])

# Check data types
print("\nData types of columns with NAs:")
for col in missing_cols:
    print(f"{col}: {X[col].dtype}")
    print(f"  Unique values: {X[col].unique()[:10]}")

Columns with missing values: ['cat_EleKtroKeuken', 'cat_Houtpelletskolen_briketten', 'cat_Incontinentie_luiers', 'cat_VNCKalfStoofvlees', 'cat_VNCRest', 'cat_Zomerspeelgoed']

Rows with missing values:
        Collishop_customer  cat_AP_STDR_PortoONLINE  cat_AP_STDR_WhiskyONLINE  \
14231                  0.0                      0.0                       0.0   
15007                  1.0                      0.0                       0.0   
15673                  1.0                      0.0                       0.0   
17726                  1.0                      0.0                       0.0   
17908                  0.0                      0.0                       0.0   
18505                  1.0                      0.0                       0.0   
20312                  1.0                      0.0                       0.0   
50663                  0.0                      0.0                       0.0   
55533                  1.0                      0.0                  

In [29]:
# Check the ORIGINAL data before any transformations
print("NAs in ORIGINAL df (before any processing):")
print(df[['cat_EleKtroKeuken', 'cat_Houtpelletskolen_briketten', 
          'cat_Incontinentie_luiers', 'cat_VNCKalfStoofvlees', 
          'cat_VNCRest', 'cat_Zomerspeelgoed']].isna().sum())

print("\n" + "="*50)

# Check after dropping target
print("\nNAs in X (after dropping target):")
X_temp = df.drop(columns=["bought_highbrow_wines"])
print(X_temp[['cat_EleKtroKeuken', 'cat_Houtpelletskolen_briketten', 
              'cat_Incontinentie_luiers', 'cat_VNCKalfStoofvlees', 
              'cat_VNCRest', 'cat_Zomerspeelgoed']].isna().sum())

print("\n" + "="*50)

# Check data types
print("\nData types in original df:")
print(df[['cat_EleKtroKeuken', 'cat_Houtpelletskolen_briketten']].dtypes)

NAs in ORIGINAL df (before any processing):
cat_EleKtroKeuken                 0
cat_Houtpelletskolen_briketten    0
cat_Incontinentie_luiers          0
cat_VNCKalfStoofvlees             0
cat_VNCRest                       0
cat_Zomerspeelgoed                0
dtype: int64


NAs in X (after dropping target):
cat_EleKtroKeuken                 0
cat_Houtpelletskolen_briketten    0
cat_Incontinentie_luiers          0
cat_VNCKalfStoofvlees             0
cat_VNCRest                       0
cat_Zomerspeelgoed                0
dtype: int64


Data types in original df:
cat_EleKtroKeuken                 float64
cat_Houtpelletskolen_briketten    float64
dtype: object


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.25,
    stratify=y,
    random_state=42
)

